Устанавливаем Lark

In [40]:
pip install lark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from lark import Lark
import xml.etree.ElementTree as ET
import lark


Ниже парсер Ларк, который принимает каждую строчку и выдает дерево по словарному входу.
parser - это грамматика, которую я прописал.

In [29]:
parser = Lark(r"""
        ?start: entry
        entry: form gramgrp? (hom|sense)* cit?

        form: orth orth_n2?
        orth: _B HEADWORD HWNUMBER? ","? _B? | _B HEADWORD "//" HEADWORD _B
        orth_n2: (_B "[" EXT "]" ","? _B)
        gramgrp: _B? "[" gen ";" number "]" ","* tns ";" gram*| "[<b>мн.</b>" number "]," tns ";" gen ";" gram*
                | tns ";" gen ";" gram* | case ";" number ";" |number? |_B? "[" gen ";" number "]," tns ";"
        case.1: [_B|","] CASE_NOUN _B // генитив, эргатив
        gen: GRAM _B | _B _I GRAM _I _B // муж, жен, III (не меняется по родам)
        number: _B GRAM | ","? GRAM _B | (_B|GRAM)+ | "," CASE_NOUN | _B GRAM _B
        tns: ","? _B* CASE_NOUN _B
        hom: pos* (_DIGIT ".") pos* (case ";" number ";")* lbl* def* extent* xr* "."?
        sense: pos* (_DIGIT ")")* (lbl* def extent* xr* | xr| lbl* usg def extent) "."?
        gram.1: _I GRAMVERB _I
        lbl: (_I WORD _I)+
        usg: _B EXT _B
        def: WORD (_I WORD _I WORD)?
        extent: _B EXT _B (_I WORD _I)? WORD
        xr.1: _I WORD _I _B EXT _B
        cit: "♦" extent* lbl* def*
        pos.1: _I PARTS0FSPEECH _I
        PARTS0FSPEECH: "союз" | "нареч." | "масд." | "частица" | "числ." | "предлог" | "в знач. сущ." | "межд."
        HEADWORD: /[\/А-Я-IΙ()◌́]+/
        HWNUMBER: /[0-9]+/
        GRAM: /[-а-яIΙ~\.\s\/()\,]+/
        WORD: /[–\/А-Яа-яё()\s,-\.;!?:«»◌́]+/
        EXT: /[–\/А-Яа-яёIΙ()~\s,-;!?\.«»◌́]+/
        _DIGIT: /[0-9]+/
        CASE_NOUN: /[-а-я\s\/\,()]+/
        GRAMVERB: "сов." | "несов."
        _B: "<b>"|"</b>"
        _I: "<i>"|"</i>"

        // -------- Importing basic data types and ignoring whitespaces --------
        %import common.WS
        %ignore WS

    """, parser="earley")



In [30]:
'''
  Функция обхода дерева lark
'''

def convert_to_xml(lark_tree_):
  if type(lark_tree_) is lark.lexer.Token:
    return lark_tree_.value
  root = ET.Element(lark_tree_.__dict__['data'].value) # Объявляем корневой тег
  for clave in lark_tree_.__dict__['children']: # Обходим ветви дерева
    trs = convert_to_xml(clave) # Рекурсивный вызов функции
    if type(trs) is str: # Если тег конечный, добавляем его как текст
      root.text = str(root.text or '') + ' ' + trs
    else:
      root.append(trs)

  return root

Теперь загружаем в ФАЙЛЫ (слева) наш текстовый файл со словарными записями, открываем и парсим его, конвертируем строки в формат тегов xml

In [31]:
with open('/content/test.txt', 'r') as rw:
  file_dict = rw.readlines()

body = ET.Element('body')
for row in file_dict:
  try:
    tree = parser.parse(row.strip()) # Парсим строку
    row_xml = convert_to_xml(tree) # Конвертируем строку в  xml
    body.append(row_xml) # Добавляем строку в общий xml
  except:
    continue
text = ET.Element('text')
text.append(body)

In [32]:
ET.indent(text, space = '    ')

In [33]:
ET.dump(text)

<text>
    <body>
        <entry>
            <form>
                <orth> А 2</orth>
            </form>
            <gramgrp />
            <sense>
                <xr> пов. накл. от эс</xr>
            </sense>
            <sense>
                <def> сказать; скажи; передай;</def>
                <extent> хΙуни ~ ты скажи.</extent>
            </sense>
        </entry>
        <entry>
            <form>
                <orth> АБА</orth>
            </form>
            <gramgrp>
                <case>  -ли, -ла</case>
                <number> абни</number>
            </gramgrp>
            <sense>
                <def> мать, мама, родительница;</def>
                <xr> см. неш</xr>
            </sense>
        </entry>
        <entry>
            <form>
                <orth> АБААЛА</orth>
            </form>
            <gramgrp>
                <case>  -ли, -ла</case>
                <number> -би</number>
            </gramgrp>
            <sense>
                <def> достат

In [34]:
teiHeader = '''
<TEI>
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Даргинско-русский словарь</title>
            </titleStmt>
            <publicationStmt>
                <p>Школа лингвистики НИУ ВШЭ</p>
            </publicationStmt>
            <sourceDesc>
                <p>Создан из словаря в формате DOC</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
</TEI>
'''

In [35]:
TEI = ET.fromstringlist(teiHeader)

In [36]:
TEI.append(text)

In [37]:
ET.dump(TEI)

<TEI>
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Даргинско-русский словарь</title>
            </titleStmt>
            <publicationStmt>
                <p>Школа лингвистики НИУ ВШЭ</p>
            </publicationStmt>
            <sourceDesc>
                <p>Создан из словаря в формате DOC</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
<text>
    <body>
        <entry>
            <form>
                <orth> А 2</orth>
            </form>
            <gramgrp />
            <sense>
                <xr> пов. накл. от эс</xr>
            </sense>
            <sense>
                <def> сказать; скажи; передай;</def>
                <extent> хΙуни ~ ты скажи.</extent>
            </sense>
        </entry>
        <entry>
            <form>
                <orth> АБА</orth>
            </form>
            <gramgrp>
                <case>  -ли, -ла</case>
                <number> абни</number>
            </gramgrp>
         

In [38]:
dict_xml_darg = ET.tostring(TEI, encoding="utf-8", method="xml").decode(encoding="utf-8") # Переводим формат (ET.element - сродни формату BeatifoulSoup) в str

In [39]:
# Сохраняем в xml-файл
fp = open('dictDarg.xml', 'w')
fp.write(dict_xml_darg)
fp.close()